# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание проекта 

В добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину. 
Шаги для выбора локации обычно такие:
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
Строят модель для предсказания объёма запасов в новых скважинах;
Выбирают скважины с самыми высокими оценками значений;
Определяют регион с максимальной суммарной прибылью отобранных скважин.
Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны.
Необходимо:
Построить модель для определения региона, где добыча принесёт наибольшую прибыль.
Проанализировать возможную прибыль и риски техникой Bootstrap.
Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv.

/datasets/geo_data_1.csv.

/datasets/geo_data_2.csv.

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек;

product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, - поскольку объём указан в тысячах баррелей.
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Загрузка и подготовка данных

In [1]:
#загрузим потенциально необходимые библиотеки,пакеты и классы.

import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from numpy.random import RandomState

In [2]:

try:
    data_0 = pd.read_csv("/datasets/geo_data_0.csv")
    data_1 = pd.read_csv("/datasets/geo_data_1.csv")
    data_2 = pd.read_csv("/datasets/geo_data_2.csv")
except:
    print("Есть ошибки")
else:
    print("Все загрузилось!")

Проанализируем информацию 

In [3]:
data_0.info()

In [4]:
data_1.info()

In [5]:
data_2.info()

In [6]:
# посмотрим на размер датасета
data_0.shape, data_1.shape, data_2.shape

Проверим на дубликаты

In [7]:
data_0.duplicated().sum()
data_1.duplicated().sum()
data_2.duplicated().sum()

Посмотрим распределения

In [8]:
data_0.hist()
plt.show()

In [9]:
data_1.hist()
plt.show()

In [10]:
data_2.hist()
plt.show()

Во всех трёх датасетах данные не вызывают вопросов по распределениям.

Построим матрицу корреляций

In [11]:
data_0\
    .corr()\
    .style\
    .background_gradient(cmap="Spectral" )

In [12]:
data_1\
    .corr()\
    .style\
    .background_gradient(cmap="Spectral" )

In [13]:
data_2\
    .corr()\
    .style\
    .background_gradient(cmap="Spectral" )

## Обучение и проверка модели

In [14]:
# Метод обучения модели и прогнозирования
def line_model(data):
    # зададим целевой признак 
    target = data['product']
    features = data.drop(['product', 'id'], axis=1)
    
    # разбьём данные на обучающую и валидационную выборки в соотношении 75:25
    features_train, features_valid, target_train,target_valid = \
    train_test_split(features, target, test_size=0.25, random_state=12345)
    
    # модель линейной регрессии
    model = LinearRegression()
    
    # обучим модель
    model.fit(features_train, target_train)
    
    # сделаем прогнозы
    predicted_valid = model.predict(features_valid)
    
    # посчитаем rmse
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    # посчитаем средний запас сырья
    predicted_valid_mean = predicted_valid.sum()/len(predicted_valid)
    
    return target_valid, predicted_valid, rmse, predicted_valid_mean

In [15]:
# получим основные показатели для каждого региона
target_valid_0, predicted_valid_0, rmse_0, predicted_valid_mean_0 = line_model(data_0)
target_valid_1, predicted_valid_1, rmse_1, predicted_valid_mean_1 = line_model(data_1)
target_valid_2, predicted_valid_2, rmse_2, predicted_valid_mean_2 = line_model(data_2)

In [16]:
print('Регион_0. Средний запас сырья:', predicted_valid_mean_0, 'rmse:', rmse_0)
print('Регион_1. Средний запас сырья:', predicted_valid_mean_1, 'rmse:', rmse_1)
print('Регион_2. Средний запас сырья:', predicted_valid_mean_2, 'rmse:', rmse_2)

Вывод: 

- Мы разделили данные регионов на обучающую и валидационную выборки.
- Обучили модель линейной регрессии на тренировочных данных регионов.
- Посчитали среднее количество предсказанных запасов в регионах, и посчитали среднеквадратическую ошибку.


- Регион_0. Средний запас сырья: 92.59256778438035 rmse: 37.5794217150813
- Регион_1. Средний запас сырья: 68.728546895446 rmse: 0.893099286775617
- Регион_2. Средний запас сырья: 94.96504596800489 rmse: 40.02970873393434

Для проверки корреляции признаков мы построили тепловые карты данных по регионам.


## Подготовка к расчёту прибыли

In [17]:
# ключевые значения для расчётов

REGION_BUDGET_RUB = 10*10**9 # бюджет на разработку 200 скважин в регионе
TOTAL_QUANTITY_POINTS = 500 # при разведке региона исследуют 500 точек
SELECTED_QUANTITY_POINTS = 200 # с помощью машинного обучения выбираем 200 точек лучших для разработки
BUDGET_QUANTITY_POINTS_RUB = REGION_BUDGET_RUB / SELECTED_QUANTITY_POINTS
BARREL_INCOME_RUB = 450 # доход с каждого барреля
PRODUCT_UNITS_INCOME_RUB = 450 * 10**3 # доход с каждой единицы продукта
LOSS_PROBABILITY_THRESHOLD_PERSENT = 2.5 # порог вероятности убытков

In [18]:
# рассчитайем достаточный объём сырья для безубыточной разработки новой скважины

min_volume_materials = REGION_BUDGET_RUB / SELECTED_QUANTITY_POINTS / BARREL_INCOME_RUB
print(f'Достаточный объём сырья для безубыточной \
разработки новой скважины равен {int(min_volume_materials)} барреля \
или {round(min_volume_materials/1000, 3)} единицы продукта (тыс. баррелей)')

In [19]:
# отклонение от среднего в регионе_0
predicted_valid_mean_0*1000 - min_volume_materials

In [20]:
# отклонение от среднего в регионе_1
predicted_valid_mean_1*1000 - min_volume_materials

In [21]:
# отклонение от среднего в регионе_2
predicted_valid_mean_2*1000 - min_volume_materials

In [22]:
print('Средняя расчетная прибыль региона_0',\
      SELECTED_QUANTITY_POINTS *\
      predicted_valid_mean_0 *\
     PRODUCT_UNITS_INCOME_RUB\
     - REGION_BUDGET_RUB)
print('Средняя расчетная прибыль региона_1',\
      SELECTED_QUANTITY_POINTS *\
      predicted_valid_mean_1 *\
     PRODUCT_UNITS_INCOME_RUB\
     - REGION_BUDGET_RUB)
print('Средняя расчетная прибыль региона_2',\
      SELECTED_QUANTITY_POINTS *\
      predicted_valid_mean_2 *\
     PRODUCT_UNITS_INCOME_RUB\
     - REGION_BUDGET_RUB)

В каждом регионе средний запас сырья скважины меньше полученного значения достаточного объёма сырья на 18.5, 42.3, 16.1 тыс. баррелей.

## Расчёт прибыли и рисков 

In [23]:
# метод для приведения к типу Series
def series_type(target, pred):
    target = target.reset_index(drop=True)
    pred = pd.Series(pred)
    return target, pred

In [24]:
# приведем целевые и прогнозные значения по всем регионам к одному типу
target_valid_0, predicted_valid_0 = series_type(target_valid_0, predicted_valid_0)
target_valid_1, predicted_valid_1 = series_type(target_valid_1, predicted_valid_1)
target_valid_2, predicted_valid_2 = series_type(target_valid_2, predicted_valid_2)

In [25]:
# метод расчёта прибыли (фактическая выручка с 200 лучших по прогнозу скважин, минус инвестиции)
def total_income(target, probabilities, count, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB):
    prob_sort = probabilities.sort_values(ascending=False)
    selected = target[prob_sort.index][:count]
    return (PRODUCT_UNITS_INCOME_RUB * selected.sum() - REGION_BUDGET_RUB)

In [26]:
# метод получения основных расчётных параметров с помощью будстрепа
def bootstrap_regions(target, predicted, SELECTED_QUANTITY_POINTS, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB):
    state = RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = TOTAL_QUANTITY_POINTS, replace=True, random_state=state)
        pred_subsumple = predicted[target_subsample.index]

        values.append(total_income(target_subsample, pred_subsumple, SELECTED_QUANTITY_POINTS, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB))
    values = pd.Series(values)
    values_mean = (values.mean())
    lower = (values.quantile(q=0.025))
    upper = (values.quantile(q=0.975))
    risk = (len(values[values < 0]) / len(values) * 100)
    return values_mean, lower, upper, risk

In [27]:
#посчитаем основные параметры для трёх регионов
values_mean_0, lower_0, upper_0, risk_0  = bootstrap_regions(target_valid_0, predicted_valid_0, SELECTED_QUANTITY_POINTS, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB)
values_mean_1, lower_1, upper_1, risk_1  = bootstrap_regions(target_valid_1, predicted_valid_1, SELECTED_QUANTITY_POINTS, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB)
values_mean_2, lower_2, upper_2, risk_2  = bootstrap_regions(target_valid_2, predicted_valid_2, SELECTED_QUANTITY_POINTS, PRODUCT_UNITS_INCOME_RUB, REGION_BUDGET_RUB)

In [28]:
print('Средняя прибыль лучших месторождений региона_0 равна', values_mean_0)
print('Средняя прибыль лучших месторождений региона_1 равна', values_mean_1)
print('Средняя прибыль лучших месторождений региона_2 равна', values_mean_2)

In [29]:
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_0', lower_0, upper_0)
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_1', lower_1, upper_1)
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_2', lower_2, upper_2)

In [30]:
print('Риск убытков региона_0 равен', risk_0, '%')
print('Риск убытков региона_1 равен', risk_1, '%')
print('Риск убытков региона_2 равен', risk_2, '%')

Выводы:
В рамках исследования проведена подготовка данных, обучена модель линейной регрессии, подготовлен прогноз запасов для скважин трёх регионов.

 - Регион_0. Средний запас сырья: 92.59256778438035 rmse: 37.5794217150813
 - Регион_1. Средний запас сырья: 68.728546895446 rmse: 0.893099286775617
 - Регион_2. Средний запас сырья: 94.96504596800489 rmse: 40.02970873393434
 

Достаточный объём сырья для безубыточной разработки новой скважины равен 111111 барреля или 111.111 единицы продукта (тыс. баррелей)

В каждом регионе средний запас сырья скважины меньше полученного значения достаточного объёма сырья на 18.5, 42.3, 16.1 тыс. 
баррелей.

Применили технику Bootstrap с 1000 выборок, для нахождения распределение прибыли:

 - Средняя прибыль лучших месторождений региона_0: 425 938 526 руб.
 - Средняя прибыль лучших месторождений региона_1: 515 222 773 руб.
 - Средняя прибыль лучших месторождений региона_2: 435 008 362 руб.

 - 95% доверительный интервал для средней прибыли 200 лучших месторождений региона_0 -102090094 947976353
 - 95% доверительный интервал для средней прибыли 200 лучших месторождений региона_1 68873225 931547591
 - 95% доверительный интервал для средней прибыли 200 лучших месторождений региона_2 -128880547 969706954

 - Риск убытков региона_0 равен 6.0 %
 - Риск убытков региона_1 равен 1.0 %
 - Риск убытков региона_2 равен 6.4 %


Регион 1 является наиболее привлекательным для разработки, регион рекомендован к освоению: высокая прибыль лучших месторождений,более узкие рамки доверительного интервала и наименьший риск получения убытков.
